In [9]:
from pathlib import Path
from joblib import Parallel, delayed

from contarg.normgrid import get_prob_vine, setup_uncert_sims, run_clusters

In [10]:
plot=True
magne_min_percentile = 99.9
uncertainty_fwhm = 2
nbootstraps = 100
block_length = 45
pairwise_sig_thresh = 0.1
sw_thresh = 2
mt_thresh = 60
make_plots=True
stimroi = "expandedcoleBA46"
refroi= "bilateralfullSGCsphere"
min_angle_motor_thresh = 30
maxMT=80
dist_std=2
scalp_res = 2
angle_std=2.5
distancetoscalp=2
overwrite = False


subjects = ['24546', '24563', '24573', '24704', '24718', '24740', '24742']
sessions = [1,2]
smoothings = [2.55, 5]
fd_thresh = 0.3

outdir = Path('/data/EDB/TMSpilot/derivatives/contarg_liston_ohbm2024')
outdir.mkdir(exist_ok=True)
jobids = {}

liston_root = Path('/data/EDB/TMSpilot/liston')

# Run Headmodel

In [11]:
# build headmodels
cmds = []
for hmsub in subjects:
    session = 1
    anat_dir = outdir / 'anat_preproc'
    ses_out_dir = anat_dir /f'sub-{hmsub}'
    ses_out_dir.mkdir(exist_ok=True, parents=True)
    proced_anat_dir = liston_root / f'sub-{hmsub}/anat'
    t1w_path = proced_anat_dir / 'T1w/T1w_acpc_dc_restore.nii.gz'
    t2w_path = proced_anat_dir / 'T1w/T2w_acpc_dc_restore.nii.gz'
    hm_outdir = ses_out_dir / f"HeadModel/m2m_{hmsub}"
    #if not hm_outdir.exists():
    print(hm_outdir)
    cmd = [
        "contarg",
        "tans",
        "headmodel",
        "--t1",
        t1w_path.as_posix(),
        "--t2",
        t2w_path.as_posix(),
        "--out-dir",
        ses_out_dir.as_posix(),
        "--subject",
        hmsub
    ]
    cmds.append(' '.join(cmd))

/data/EDB/TMSpilot/derivatives/contarg_liston_ohbm2024/anat_preproc/sub-24546/HeadModel/m2m_24546
/data/EDB/TMSpilot/derivatives/contarg_liston_ohbm2024/anat_preproc/sub-24563/HeadModel/m2m_24563
/data/EDB/TMSpilot/derivatives/contarg_liston_ohbm2024/anat_preproc/sub-24573/HeadModel/m2m_24573
/data/EDB/TMSpilot/derivatives/contarg_liston_ohbm2024/anat_preproc/sub-24704/HeadModel/m2m_24704
/data/EDB/TMSpilot/derivatives/contarg_liston_ohbm2024/anat_preproc/sub-24718/HeadModel/m2m_24718
/data/EDB/TMSpilot/derivatives/contarg_liston_ohbm2024/anat_preproc/sub-24740/HeadModel/m2m_24740
/data/EDB/TMSpilot/derivatives/contarg_liston_ohbm2024/anat_preproc/sub-24742/HeadModel/m2m_24742


In [12]:
swarm_cmd_dir = anat_dir/'swarm/swarm_cmds'
swarm_cmd_dir.mkdir(exist_ok=True, parents=True)
swarm_log_dir = anat_dir/'swarm/swarm_log'
swarm_log_dir.mkdir(exist_ok=True, parents=True)

In [13]:
if len(cmds) > 0:
    swarm_cmd_file = swarm_cmd_dir / 'headmodels'
    
    swarm_cmd_file.write_text('\n'.join(cmds))
    run_name = 'headmodels'
    jobids[run_name] = ! swarm -f {swarm_cmd_file} -g 80 -t 22 --module matlab,freesurfer/6,fsl,simnibs/4.0,connectome-workbench,ANTs  --time 12:00:00 --logdir {swarm_log_dir} --job-name {run_name} --partition norm
    print(jobids[run_name])

['13961288']


# Run simulations for each point on the gyral lip

In [ ]:
subjects = ['24546', '245461', '245462']
cmds = []
for subject in subjects:
    headmodel_dir = anat_dir /f'sub-{subject}/HeadModel'
    searchgrid_dir = anat_dir /f'sub-{subject}/SearchGrid'
    out_dir = anat_dir /f'sub-{subject}/Simulation'
    src_surf_dir = liston_root / f'sub-{subject}/anat/T1w/fsaverage_LR32k/'

    cmd = [
        'contarg',
        'normgrid',
        'sim-gyral-lip',
        '--headmodel-dir',
        headmodel_dir.as_posix(),
        '--searchgrid-dir',
        searchgrid_dir.as_posix(),
        '--out-dir',
        out_dir.as_posix(),
        '--src-surf-dir',
        src_surf_dir.as_posix(),
        '--njobs',
        "20"
    ]
    cmds.append(' '.join(cmd))

In [ ]:
liston_dir = Path('/data/EDB/TMSpilot/liston/')
swarm_cmd_dir = liston_dir/'swarm/swarm_cmds'
swarm_cmd_dir.mkdir(exist_ok=True, parents=True)
swarm_log_dir = liston_dir/'swarm/swarm_log'
swarm_log_dir.mkdir(exist_ok=True, parents=True)

In [ ]:
if len(cmds) > 0:
    swarm_cmd_file = swarm_cmd_dir / 'gyral_opts'
    swarm_cmd_file.write_text('\n'.join(cmds))
    run_name = 'gyral_opts'
    jobid = ! swarm -f {swarm_cmd_file} -g 125 -t 22 --gres=lscratch:100 --module matlab,freesurfer/6.0,fsl,simnibs/4.0,connectome-workbench,openblas  --time 4:00:00 --logdir {swarm_log_dir} --job-name {run_name} --partition quick,norm
    print(jobid)

# Run simulations for positional uncertainty

## set up dataframes with parameters for uncertainty runs

In [ ]:
anat_dir = Path('/data/EDB/TMSpilot/derivatives/contarg_liston/anat_preproc/')


coil='MagVenture_MCF-B65.ccd'

if not Path(coil).exists():
    simnibs_coil_dir = Path(resource_filename('simnibs', 'resources/coil_models'))
    coil_path = simnibs_coil_dir / f'Drakaki_BrainStim_2022/{coil}'
    if not coil_path.exists():
        coil_path = simnibs_coil_dir / f'legacy_and_other/{coil}'
        if not coil_path.exists():
            raise FileNotFoundError(f"Could not find coil ({coil}) in {simnibs_coil_dir}.")
else:
    coil_path = Path(coil)

In [ ]:
nsims = 1000
uncert_deviations_path = anat_dir / f'uncert_deviations{nsims}.npy'
if not uncert_deviations_path.exists():
    uncert_deviations = make_uncert_sims(5, nsims, dist_std, angle_std)
    np.save(uncert_deviations_path, uncert_deviations)
vine = get_prob_vine(4, dist_std, angle_std)

In [ ]:
jobs = []
for subject in subjects:
    sim_dir = Path(anat_dir / f'sub-{subject}/Simulation/simulation-00')
    if not sim_dir.exists():
        sim_dir = Path(anat_dir / f'sub-{subject}/Simulation/simulation-01')
        if not sim_dir.exists():
            print(f"{sim_dir} not found, skipping")
    headmodel_dir = anat_dir / f'sub-{subject}/HeadModel'
    jobs.append(delayed(setup_uncert_sims)(headmodel_dir, sim_dir, dist_std=dist_std, angle_std=angle_std, outname=f'uncert{nsims}', uncert_deviations_path=uncert_deviations_path))
    
    

In [ ]:
all_settings = Parallel(n_jobs=8, verbose=10)(jobs)

In [ ]:
cmds = []
for (subject, settings_path) in zip(subjects, all_settings):
    settings = pd.read_pickle(settings_path)
    for oix in sorted(settings.oix.unique()):
        sim_dir = Path(anat_dir / f'sub-{subject}/Simulation/simulation-00')
        if not sim_dir.exists():
            sim_dir = Path(anat_dir / f'sub-{subject}/Simulation/simulation-01')
            if not sim_dir.exists():
                print(f"{sim_dir} not found, skipping")
        
        HeadModel_dir = anat_dir / f'sub-{subject}/HeadModel'
        try:
            m2m_dir = sorted(HeadModel_dir.glob('m2m*'))[0]
        except IndexError:
            raise FileNotFoundError(f"No m2m directory found in {HeadModel_dir}")
        headmesh_path = m2m_dir / f'{subject}.msh'
        
        uncert_dir = settings_path.parent
        uncert_out = uncert_dir / f'oix-{oix:04d}_stat-abovethreshactprobs_magnE.nii.gz'
        if not uncert_out.exists() or overwrite:
            cmd = [
                'contarg',
                'normgrid',
                'sim-uncert',
                '--headmesh-path',
                headmesh_path.as_posix(),
                '--settings-path',
                settings_path.as_posix(),
                '--ix',
                f'{oix}',
                '--coil-path',
                coil_path.as_posix(),
                '--tmp-dir',
                f'/lscratch/$SLURM_JOB_ID/uncertsim-{oix}',
                '--out-dir',
                settings_path.parent.as_posix(),
                '--njobs',
                f'20'
            ]
            cmds.append(' '.join(cmd))

In [ ]:
# deal with 1000 job swarm limit
if len(cmds) > 0:
    swarm_cmd_file = swarm_cmd_dir / 'uncert_sims'
    swarm_cmd_file.write_text('\n'.join(cmds[:1000]))
    run_name = 'uncert_sims'
    jobid = ! swarm -f {swarm_cmd_file} -g 100 -t 22 --gres=lscratch:400 --module matlab,freesurfer/6.0,fsl,simnibs/4.0,connectome-workbench,openblas  --time 8:00:00 --logdir {swarm_log_dir} --job-name {run_name} --partition norm
    print(jobid)

In [ ]:
if len(cmds) > 1000:
    swarm_cmd_file = swarm_cmd_dir / 'uncert_sims'
    swarm_cmd_file.write_text('\n'.join(cmds[1000:]))
    run_name = 'uncert_sims'
    jobid = ! swarm -f {swarm_cmd_file} -g 100 -t 22 --gres=lscratch:400 --module matlab,freesurfer/6.0,fsl,simnibs/4.0,connectome-workbench,openblas  --time 8:00:00 --logdir {swarm_log_dir} --job-name {run_name} --partition norm
    print(jobid)

## Here's code for restarting failues

In [ ]:
to_restart = [cmds[164], cmds[222], cmds[226], cmds[360]]
to_restart

In [ ]:
if len(to_restart) > 0:
    swarm_cmd_file = swarm_cmd_dir / 'retart_uncert_sims'
    swarm_cmd_file.write_text('\n'.join(to_restart))
    run_name = 'restart_uncert_sims'
    jobid = ! swarm -f {swarm_cmd_file} -g 100 -t 22 --gres=lscratch:400 --module matlab,freesurfer/6.0,fsl,simnibs/4.0,connectome-workbench,openblas  --time 4:00:00 --logdir {swarm_log_dir} --job-name {run_name} --partition quick,norm
    print(jobid)

# Run repdist clustering on time series so we can get stats for each position

This can be run in parallel with the gyral lip and uncertainty simulations above 

In [ ]:
jobs = []
clusts = []
for subject in subjects:    
    for session in sessions:
        for smoothing in smoothings:
            subj_func_outdir =  outdir / f'func_preproc/sub-{subject}/'
            ses_func_outdir = subj_func_outdir /  f'ses-{session}'
            clust_outdir = ses_func_outdir / 'cluster'
            lrest_dir = liston_root / f'sub-{subject}/func/rest'
            concat_nii = lrest_dir / f'session_{session}/concatenated/Rest_session-{session}_OCME+MEICA+MGTR_Concatenated+SubcortRegression+SpatialSmoothing{smoothing}.dtseries.nii'
            src_surf_dir = liston_root / f'sub-{subject}/anat/T1w/fsaverage_LR32k'
            jobs.append(delayed(run_clusters)(
                subject,
                concat_nii,
                clust_outdir,
                src_surf_dir,
                out_prefix=f'sub-{subject}_ses-{session}_smoothing-{smoothing}_',
                overwrite=True
            ))
            clusts.append(dict(
                subject = subject,
                session = session,
                smoothing = smoothing
            ))
            

In [ ]:
clustreses = Parallel(n_jobs=20, verbose=10)(jobs)